# 02. Feature Engineering & Advanced Preprocessing

## 1. Project Objectives
Following the initial data exploration, this notebook focuses on transforming raw attributes into high-signal features. We will address the **22.5% class imbalance** and construct a robust preprocessing pipeline for production-ready modeling.

### 1.1 Key Strategies
* **Domain-Specific Ratios**: Creating financial indicators (e.g., Loan-to-Income) to better quantify risk.
* **Imbalance Management**: Integrating **SMOTE** (Synthetic Minority Over-sampling Technique) to balance the learning surface.
* **Engineering Rigor**: Using `imblearn.pipeline` to ensure resampling is only applied to training data, preventing data leakage.
* **Stratification**: Maintaining target distribution across train/test splits.

In [1]:
import os 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer

# Note: We use imblearn's Pipeline to handle resampling correctly
from imblearn.pipeline import Pipeline 
from imblearn.over_sampling import SMOTE

# Plotting config
plt.style.use('ggplot')
np.random.seed(42)

In [2]:
# Define absolute paths for portability
DATA_DIR = r"C:\dev\quant_project\homework\data_storage"
MODEL_DIR = r"C:\dev\quant_project\homework\model_storage"

# Ensure directories exist
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

In [3]:
# Load data
try:
    df = pd.read_csv(os.path.join(DATA_DIR, 'loan_data.csv'))
    print(f"✅ Data loaded. Default Rate: {df['default'].mean():.2%}")
except FileNotFoundError:
    print("❌ 'loan_data.csv' not found. Please run Part 01 first.")

✅ Data loaded. Default Rate: 22.48%


## 2. Advanced Feature Engineering

In credit risk modeling, the relationship between income, debt, and loan size is often more predictive than individual raw values. We will construct features that act as proxies for a borrower's **Financial Cushion**.

### 2.1 Engineered Features:
1.  **Loan-to-Income Ratio (LTI)**: Quantifies the total debt burden relative to annual earnings.
2.  **Revenue Coverage**: Measures how many years of monthly revenue would be required to cover the requested loan.
3.  **Credit Tiers**: Discretizing the credit score to capture non-linear risk thresholds (e.g., the jump in risk from 'Subprime' to 'Average').

In [4]:
def engineer_features(input_df):
    """
    Applies financial domain logic to the loan dataset.
    """
    df_eng = input_df.copy()
    
    # 1. Debt Burden Ratio
    df_eng['loan_to_income'] = df_eng['loan_amount'] / (df_eng['annual_income'] + 1e-6)
    
    # 2. Revenue-to-Loan Sustainability
    df_eng['rev_to_loan_ratio'] = (df_eng['monthly_revenue'] * 12) / (df_eng['loan_amount'] + 1e-6)
    
    # 3. Employee Efficiency Proxy
    df_eng['rev_per_employee'] = (df_eng['monthly_revenue'] * 12) / (df_eng['num_employees'] + 1)
    
    # 4. Non-linear Credit Tiers
    # Creating buckets to address the weak linear correlation found in EDA
    bins = [300, 580, 670, 740, 800, 850]
    labels = ['High_Risk', 'Subprime', 'Average', 'Good', 'Excellent']
    df_eng['credit_tier'] = pd.cut(df_eng['credit_score'], bins=bins, labels=labels)
    
    return df_eng

# Apply engineering
df_featured = engineer_features(df)
print("✅ Domain features engineered.")
display(df_featured[['loan_to_income', 'rev_to_loan_ratio', 'credit_tier']].head())

✅ Domain features engineered.


,loan_to_income,rev_to_loan_ratio,credit_tier
0,0.370241,1.179165,Average
1,0.399697,5.484552,Average
2,1.032594,5.935224,Good
3,0.783714,4.156433,Good
4,0.231562,5.830639,Good


## 3. Data Splitting (Before Resampling)

As a best practice, we split our data **before** applying any over-sampling (SMOTE). This ensures the test set remains an "untouched" representation of reality, free from synthetic noise.

In [5]:
# Define target and predictors
X = df_featured.drop(columns=['application_id', 'default'])
y = df_featured['default']

# Stratified split to maintain 22.5% default rate in both sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {X_train.shape[0]} samples (Default Rate: {y_train.mean():.2%})")
print(f"Test set: {X_test.shape[0]} samples (Default Rate: {y_test.mean():.2%})")

Training set: 4000 samples (Default Rate: 22.48%)
Test set: 1000 samples (Default Rate: 22.50%)


## 4. The Integrated Preprocessing & SMOTE Pipeline

We define a nested pipeline that handles three distinct tasks:
1.  **Imputation**: Filling missing values for `credit_score` and `monthly_revenue`.
2.  **Transformation**: Scaling numerical features and One-Hot Encoding categorical ones.
3.  **Resampling**: Applying SMOTE to the training data to re-balance the class distribution.

In [6]:
# Identify feature types
numeric_features = [
    'annual_income', 'credit_score', 'loan_amount', 'years_in_business', 
    'num_employees', 'previous_loans', 'debt_to_income_ratio', 
    'monthly_revenue', 'loan_to_income', 'rev_to_loan_ratio', 'rev_per_employee'
]

categorical_features = [
    'loan_purpose', 'industry', 'has_collateral', 
    'education_level', 'geographic_region', 'credit_tier'
]

# 1. Numerical Pipeline: Median Imputation + Scaling
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# 2. Categorical Pipeline: Constant Imputation + One-Hot Encoding
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# 3. Combine into a central Preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, numeric_features),
        ('cat', cat_transformer, categorical_features)
    ]
)

# 4. Final Training Pipeline (Preprocessing + Resampling)
# We target a 0.5 ratio (Minority = 50% of Majority) to balance signal without overwhelming the data with synthetic noise.
resampling_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('smote', SMOTE(random_state=42, sampling_strategy=0.5))
])

print("✅ Preprocessing and SMOTE Pipeline successfully defined.")

✅ Preprocessing and SMOTE Pipeline successfully defined.


## 5. Post-Resampling Feature Validation

Before modeling, we verify that SMOTE has preserved (or enhanced) the risk signals identified in Part 01. We analyze the correlation on the **resampled training set**.

In [7]:
# Manually fit and transform training data to check resampled correlations
# Note: This is for analysis only; the pipeline handles this automatically during training
X_train_processed = preprocessor.fit_transform(X_train)
sm = SMOTE(random_state=42, sampling_strategy=0.5)
X_res, y_res = sm.fit_resample(X_train_processed, y_train)

# Extract feature names for the analysis
cat_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
ohe_feature_names = cat_encoder.get_feature_names_out(categorical_features)
all_feature_names = numeric_features + list(ohe_feature_names)

# Create a validation dataframe
df_resampled = pd.DataFrame(X_res, columns=all_feature_names)
df_resampled['default'] = y_res

# Compare correlation with Part 01
new_corr = df_resampled.corr()['default'].sort_values()
print("--- Top Drivers after Resampling ---")
print(new_corr.head(3)) # Negative drivers
print(new_corr.tail(3)) # Positive drivers

--- Top Drivers after Resampling ---
annual_income        -0.382605
rev_to_loan_ratio    -0.126429
has_collateral_yes   -0.111808
Name: default, dtype: float64
loan_to_income    0.347064
loan_amount       0.350249
default           1.000000
Name: default, dtype: float64


## 6.1 Insights from Post-Resampling Validation

The correlation analysis on the resampled training set provides several critical insights into our feature set and the effectiveness of our balancing strategy:

### 1. Feature Engineering Validation
* **High-Impact Ratios**: The engineered feature `loan_to_income` (0.347) has emerged as one of the top predictors of default. This validates our domain-driven approach—measuring the relative burden of debt is more telling than looking at `loan_amount` or `annual_income` in isolation.
* **Cash Flow Signal**: `rev_to_loan_ratio` (-0.126) shows a strong negative correlation, confirming that businesses with higher revenue coverage relative to their debt are significantly lower risk.

### 2. Economic Logic Alignment
* **Income & Loan Size**: `annual_income` (-0.382) and `loan_amount` (0.350) remain the "anchor" features. The model now has a much clearer decision boundary between high-income borrowers and high-debt seekers.
* **Collateral Impact**: The presence of collateral (`has_collateral_yes`: -0.111) is a significant risk-mitigating factor, which aligns with standard credit risk theory.

### 3. The "SMOTE Amplifier" Effect
* Compared to the raw EDA in Part 01, the correlation coefficients have become more pronounced. By balancing the classes, we have reduced the "noise" created by the majority class and allowed the underlying risk signals of the minority class (defaulters) to surface.

### 4. Strategic Implications for Modeling
* Since the signals are now statistically stronger, our champion models (like XGBoost) will likely achieve higher **Recall** without sacrificing too much **Precision**.
* **Observation on Credit Score**: While `credit_score` is not in the top 3, the financial ratios we built (incorporating income and revenue) have taken over as the primary drivers, providing a more holistic view of "Ability to Pay" vs. just "Willingness to Pay."

## 6. Summary of Part 2

### Key Achievements:
* **Leakage Prevention**: All transformations are encapsulated in a pipeline, ensuring the test set remains strictly independent.
* **Domain Expertise**: Added `loan_to_income` and `rev_to_loan` ratios to amplify weak raw signals.
* **Balanced Learning**: Integrated SMOTE to elevate the minority class signal, addressing the 22.5% imbalance issue.

---
**Next Step**: Proceed to `03_Model_Development_and_Evaluation.ipynb` to train Baseline and Advanced models using this pipeline.